# Azure Developer Days .NET 2024

## 🦸🏻 Help your Super Hero friend with AI 🦸🏼‍♀️

Semantic Kernel has been created to allow developers to seamlessly integrate Artificial Intelligence into their applications. To achieve this, it provides a set of features that will enable you to add models, prompts, native functions, and memories without requiring deep knowledge of AI 🥲. That's why it's said that Semantic Kernel simulates the brain 🧠 of your application.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

# Environment variables

Let's start importing the keys and utils that we will need for this demo.

In [ ]:
// Azure OpenAI information
var keys = (await Microsoft.DotNet.Interactive.Kernel
    .GetPasswordAsync("Give me your Azure OpenAI information"))
    .GetClearTextPassword().Split(';');

var azureOpenAIDeploymentName = keys[0];
var azureOpenAIEndpoint = keys[1];
var azureOpenAIKey = keys[2];

In [ ]:
// get super hero api key
var superHeroApiKey = (await Microsoft.DotNet.Interactive.Kernel
    .GetPasswordAsync("Give me your Super Hero Api key")).GetClearTextPassword();

In [25]:
#load "Utils/consoleUtils.cs"

## How to get started with Semantic Kernel


The first thing you need to be able to run Semantic Kernel in this notebook is to install the **Microsoft.SemanticKernel** library, which is currently in version **1.7.1** (I promise to keep updating 🤓). 

You can find more information about this library and its updates on the official NuGet page [here](https://www.nuget.org/packages/Microsoft.SemanticKernel).



In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"

In [ ]:
using Microsoft.SemanticKernel;

// Create a Builder
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    deploymentName: azureOpenAIDeploymentName, 
    endpoint: azureOpenAIEndpoint, 
    apiKey: azureOpenAIKey);
var kernel = builder.Build();

## Using Semantic Kernel plugins

Plugins are the core of Semantic Kernel. With them, you encapsulate capabilities so that they can be reusable, maintainable, and plannable (you will understand it later 🙃). There are two types: those that consist of prompt templates called **Semantic Functions** and native functions of the chosen programming language called **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

## Semantic Functions

When you talk to artificial intelligence models, you must do so with what is known as a *prompt*. This can range from a simple phrase to something more elaborate, allowing the model to understand not only what we want but also how we want it. If you take a look at the official documentation, it defines this type of function as the mouth 👄 and ears 👂🏻 of your brain 🧠. 🤖🌐

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

As part of this repo, you have a folder called **SemanticFunctions** that has different functions of this type:

- **FunPlugin**: This allows us to ask the model to make jokes about heroes under certain conditions, through the **Joke** function.
- **WritePlugin**: To show you that within a plugin you can have different functions, in this directory we have two related to the art of writing: the first one, **OOF**, allows us to generate the "Out of Office" 🏢📧 message for superheroes and the second **StoryGen** will help us create stories, also about superheroes 🦸🏻‍♂️🦸🏻‍♀️.

For our kernel to know that these plugins are available, you first need to get the path of the directory:


In [ ]:
using System.IO;

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), 
    "SemanticFunctions", "FunPlugIn");

// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectory);

In [ ]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a April's fools joke",
    ["hero"] = "Batman"
};
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], variables);
Console.WriteLine(result.GetValue<string>());

In the same way, we can use the functions included in **WriterPlugin**:


In [ ]:
// Load the WriterPlugin from the Plugins Directory
var pluginsDirectoryWriter = Path.Combine(System.IO.Directory.GetCurrentDirectory(), 
    "SemanticFunctions", "WriterPlugin");

var writerPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectoryWriter);

var variables = new KernelArguments() {
    ["input"] = "Tell me a Spring Story",
    ["hero"] = "Superman"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["StoryGen"], variables);
Console.WriteLine(result);

Or even to create messages for when the heroes 🦸🏻‍♀️ are on vacation ✈️🚢🌴:


In [26]:
var variables = new KernelArguments() {
    ["input"] = "Create an OOF for Christmas",
    ["hero"] = "Hulk"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["OOF"], variables);
WriteLongStory(result.GetValue<string>());

Hello there!

🦸‍♂️ Hulk here, smashing through emails and saving the day!

I'm
currently out of the office to celebrate the holiday season with my friends and
family. 🎄 Don't worry, I'll be back to smash some more villains soon!

In the
meantime, if you need immediate superhero help, please contact my good friend
Thor. He's always ready to lend a helping hand...or hammer! ⚡️

Thank you for
your understanding and patience. I'll respond to your messages as soon as I
return!

👊 Stay incredible and have a smashing good time! 👊

Best
regards,
Hulk


## Native Functions

While semantic functions allow us to define and reuse prompts, **with native functions you can make the semantic kernel call functions written in C# or Python**, for tasks that go beyond a call through a prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

For this example, I'm going to use an API called **SuperHero API**, which requires an API key. You can get it from their website: [https://superheroapi.com/](https://superheroapi.com/)

Now, to load a native function, we must add this before we build the Kernel.

Let's create a new Super Hero Info class and add it as a Plugin.


In [ ]:
#load "NativeFunctions/HeroInfo.cs"

var heroInfo = new HeroInfo(superHeroApiKey);
builder.Plugins.AddFromObject(heroInfo, "HeroInfo");
Kernel kernel = builder.Build();

The builder now can interact with the local function. Let's make a call to get the alter ego of a super hero.

In [ ]:
var kernelArgs = new KernelArguments()
{
    ["input"] = "Wonder Woman"
};

var result = await kernel.InvokeAsync<string>("HeroInfo", "GetAlterEgo", kernelArgs);
Console.WriteLine(result);

## Planner

So far, all the plugins you have seen have been executed intentionally. That is, no one has chosen them for you and you can run them based on your needs. However, this is the most *static* way to interact with Semantic Kernel. There is another option called **Planner** that will leave you astounded 😮

Planner is a function that takes a user's request and returns a plan on how to carry out the request. To do this, it uses AI to combine the plugins registered in the core and recombine them into a series of steps that complete a goal.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview"

Now we are ready to create a planner.

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();

In [ ]:
var ask = @"I would like you to tell me a joke about Batman, " + 
"and with that joke, create an out-of-office message using the joke.";

#pragma warning disable SKEXP0060
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

In [ ]:
// executing the plan
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
WriteLongStory(originalPlanResult.ToString());

### Advanced Scenario

The kernel already has the plugins loaded, and the function, let's see if it can solve a more complex example:


In [ ]:
var ask_complex = @"I would like you to find out IronMan's alter ego," + 
" and create an out of office message for IronMan, signing it with his alter ego.";

#pragma warning disable SKEXP0060
var plan_complex = await planner.CreatePlanAsync(kernel, ask_complex);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan_complex);

In [ ]:
#pragma warning disable SKEXP0060
var complexPlanResult = await plan_complex.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Complex  Plan results:\n");
WriteLongStory(complexPlanResult.ToString());